<img src="../files/misc/logo.png" width=300/>
<h1 style="color:#872325"> Webscrapping </h1>

De acuerdo a [Wikipedia](https://es.wikipedia.org/wiki/Web_scraping), *Web Scrapping*, 

> es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la World Wide Web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

Para fines prácticos, realizamos un web scrapping cuándo no contamos con alguna manera de acceder a información estructurada de datos por analizar (.csv, .json, base de datos, API). 

---

## HTML
Al hacer webscrapping, dependemos de la estructura de la página web para poder acceder a la información que deseamos obtener. La estructura de la página web se define mediante un lenguaje llamado `HTML` (Hypertext Markup Language) el cual se compone por medio de *tags*.

```HTML
<tag>
</tag>
```

Un *tag* define una sección de la página. Cada *tag* tiene una propiedad. Un conjunto de estos tags definen el cuerpo de la página.


### Una primera página web (`../files/lec04/first.html`)
```HTML
<!DOCTYPE html>
<html>
<body>

<h1>Data Science and Machine Learning</h1>

</body>
</html>
```

Para nuestros fines, un *tag* representará una sección dentro de la página la cuál nos gustaría explorar o extraer.

---

## CSS

Un *Cascading Style Sheet* (CSS), es un archivo que viene acompañado de un html y define el estilo de la página.

```CSS
selector {
    property: property-value;
}
```

Cada *selector* contiene diferentes [propiedades](http://web.simmons.edu/~grabiner/comm244/weekthree/css-basic-properties.html) de acuerdo al *tag* que le corresponda.

## References


* https://developer.mozilla.org/en-US/docs/Web